# O q fazer
- Tentar aplicar a nova maneira de fazer previsões
- Criar um OddTime parameter (intervalo de variação da ODD desde que acabaram os últimos jogos de ambas as equipas )
- Fazer Clustering (OddTime) -> Se calhar so vai ser possível 

# NonBinary Trees
- Novo tipo de floresta que gostava de experimentar
- Onde, a formulação de árvores é automática

# NOTA:
## Parâmetros que se podem alterar
- Os thresholds para as divisões das perguntas
- O "Picked" no Column_Pick (que por agr não deixa que uma coluna seja repetida na mesma árvore)
- Se queremos que em cada "grau de uma árvore" a coluna pergunta seja a mesma ou possa variar. De momento é a mesma

In [1]:
import random
import pandas as pd
import numpy as np
import time
from functools import reduce
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',200)

In [2]:
class NB_Decision_Node:
#branches é uma lista com os vários NB_Decision_Node
    def __init__(self,question,branches): 
        self.question = question
        self.branches = branches
        self.branches_size = len(branches)
        
class NB_Leaf:
    
    def __init__(self,df):
        self.predictions=depend_counts(df) 
        self.probability=proba(df)
        
class NB_Question:
#values é tambem uma lista com valores
    def __init__(self,column,values):
        self.column = column
        self.values = values
        self.values_size = len(values)
        

class NB_Forest:
    def __init__(self):
        self.trees=[]
        
    def add_tree(self,tree):
        (self.trees)+=[tree]
        
    def size(self):
        return len(self.trees)
    
    def get_tree(self,ind):
        return self.trees[ind]
    
    def merge_forests(self,forest):
        (self.trees)+=(forest.trees)
        
def NB_partition(df,Q):
    parts = []
    for i in range(Q.values_size):
        if i==0:
            t1 = np.where(df[:,Q.column]>Q.values[i]) #vamos querer o oposto disto
            df1 = np.delete(df,t1,0)  #por isso é q aqui fazemos delete, para tirar o conjunto em cima
            #print(df1)
            parts += [df1]
        else:
            #print(Q.values[i-1],'< ||||  <=',Q.values[i])
            t1 = np.where(np.logical_or(df[:,Q.column]<=Q.values[i-1],df[:,Q.column]>Q.values[i]))
            df1 = np.delete(df,t1,0)
            
            parts += [df1]
            
    t1 = np.where(df[:,Q.column]<=Q.values[i])
    df1 = np.delete(df,t1,0)
    #print(df1)
    parts += [df1]
    return parts
            

In [3]:
def Create_Random_df(size,columns,col_delta):
    for i in range(columns):
        itn = random.randrange(col_delta[0],col_delta[1]+1)
        vars()['col'+str(i)]=itn
        vars()['C'+str(i)]=[]
    for c in range(size):
        for i in range(columns-1):
            vars()['C'+str(i)]+=[random.randrange(1,vars()['col'+str(i)]+1)]
        vars()['C'+str(i+1)]+=[random.randrange(2)]
    DF=[]
    
    for d in range(columns):
        df_ = pd.DataFrame({'Col'+str(d):vars()['C'+str(d)]})
        DF+=[df_]
    return pd.concat(DF,axis=1)

In [5]:
df = Create_Random_df(3000,50,[30,200])

In [6]:
df

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,Col41,Col42,Col43,Col44,Col45,Col46,Col47,Col48,Col49
0,147,68,38,2,6,18,61,91,83,9,77,10,43,51,146,124,45,25,15,32,56,18,22,94,195,37,106,114,34,46,25,152,19,39,1,60,99,31,13,61,20,51,44,9,35,133,30,28,33,0
1,34,164,41,78,9,3,66,69,70,50,129,37,40,105,156,9,112,83,70,38,54,28,22,103,137,77,2,130,129,5,84,135,59,76,71,102,138,15,5,11,24,5,69,121,17,4,94,2,123,1
2,16,25,54,63,98,8,68,55,107,57,19,93,119,174,30,139,109,82,103,63,62,80,36,39,51,63,149,131,138,4,134,118,34,32,19,83,75,44,109,141,35,52,85,129,23,65,105,8,141,0
3,8,64,81,88,128,1,32,11,4,5,35,63,118,58,102,143,88,3,109,3,35,100,30,99,184,11,111,75,68,6,63,178,17,52,43,75,137,14,69,102,33,33,91,21,41,86,71,42,15,1
4,147,48,70,29,25,15,66,67,30,17,13,58,80,159,153,80,113,107,28,95,37,15,13,142,89,39,30,58,32,45,4,47,41,37,63,45,116,42,50,95,11,19,88,65,44,25,114,21,68,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,128,134,35,95,84,6,48,40,115,38,40,25,99,62,137,31,42,129,53,78,3,45,8,3,45,36,116,80,154,1,139,77,29,40,12,92,95,17,2,133,44,5,59,43,44,23,45,25,143,0
2996,124,104,38,55,126,4,24,11,105,6,26,90,185,64,132,151,78,126,14,7,11,48,16,121,20,27,112,76,115,23,148,20,10,75,92,55,170,14,70,124,11,17,65,75,4,71,66,4,14,1
2997,158,18,1,12,47,34,30,111,28,43,11,68,137,64,121,54,73,136,30,10,15,40,5,127,185,35,140,89,110,46,101,159,29,15,94,31,149,37,111,24,15,44,54,76,38,139,25,20,25,1
2998,7,29,41,137,50,23,55,43,92,65,28,55,137,92,140,124,87,68,18,31,18,43,34,43,38,69,63,89,165,40,138,27,58,50,46,11,80,25,27,23,17,47,65,115,25,121,15,30,32,1


In [4]:
def Get_Question(df,c,x): 
#pega na coluna(c) e df e em quantas divisoes da arvore queremos fazer (x)
# e devolve x "ramos", com o mesmo número de elementos
# o c das colunas é números
# a pergunta
    x_size = round(len(df)/x)
    df_aux = df[df[:,c].argsort()]
    Qs=[]
    for i in range(1,x):
        Qs+=[df_aux[i*x_size][c]]
    return NB_Question(c,Qs)
    

### Divisão das colunas por nº de elementos
- 2 : [2,8]
- 3 : [9,15]
- 4 : [16,60]
- 5 : [61,...]


Aqui até podíamos ter divisores maiores, contudo, isso significa que a df original vai ser dividida em partes muito pequenas, assim chegamos rapidamente às folhas e acabamos por não relacionar muito as colunas, o que é essencial neste modelo

##### Isto vai estar numa função que poderá ser alterada
ATENÇÃO: Uma coluna com o número de valores únicos muito perto do tamanho da df original, é uma má coluna. Por isso mesmo, é que é preciso uma função que agregue esses valores únicos em grupos de modo a aumentar a diferença entre estes e o tamanho da df original

In [9]:
1500/15

100.0

In [5]:
def Get_ColDiv(n):
    #recebe o nº de el de uma coluna e diz quantas divisões vamos fazer
    if n==1:
        return False
    
    elif 2<=n<=8:
        return 2
    
    elif 9<=n<=15:
        return 3
    
    elif 16<=n<=60:
        return 4
    
    elif 61<=n:
        return 5
    
    #elif 101<=n<=250:
    #    return 8
    
    #elif n>=251:
    #    return 10
    
    else:
        print('counts da coluna',n)
        raise Exception('Valor WTF')

In [11]:
len(np.unique(df2[:,1]))

NameError: name 'df2' is not defined

In [6]:
def Column_Pick_old(df,col_list,tree_degree,picked,limit_up,limit_down,limit_count,col_count,Picked=True):
    """
    Trata do processamento da escolha das colunas
    df -> DataFrame em análise
    col_list -> lista dos índices de todas as colunas
    tree_degree -> em que "grau da árvore" estamos (começa em 0 como o python)
    picked -> lista das colunas já escolhidas na árvore
    para não repetir colunas na mesma árvore
    Picked -> True se é para ter em conta o picked, False caso contrário
    Logo é uma espécie de hyperparametro
    Limit_up -> limite de tamanho das nossas folhas (têm de ser preferencialmente abaixo deste valor)
    limit_down -> As folhas não podem de maneira alguma serem mais pequenas q este valor
    count,limit_count -> parâmetros que precisam de uma longa explicação: (o count não é argumento, mas é logo criado no ínicio*)
    Quando tivermos perto das folhas, será dificil encontrar algum valor dentro do limit_up/down
    Contudo, como o limit_down é mais importante, decidi que ao fim de limit_count tentativas
    Para encontrar uma coluna ideal, o programa desiste e devolve as dfs como estão como folhas
    Por agora não vou criar parâmetro para evitar que colunas se repitam neste último processo do count
    Relacionado com as folhas (pq em 100 colunas é difícil calhar a mesma 2x seguidas)
    E se calhar, vamos ignorar esse acontecimento
    ---------------------------------
    Esta função devolve já a pergunta a aplicar na árvore e a coluna escolida
    Pois vai ser necessário, uma vez que a minha ideia é que as colunas das perguntas sejam iguais 
    Para cada "grau da árvore"
    """
    print('Vens aqui?')
    if len(col_list)>=tree_degree+1:
        if col_list[tree_degree][2]=='Leaf':
            Get_Question(df,COL,n),'Leaf',col_list  #para diferenciar Branch e Leaf
        elif col_list[tree_degree][2]=='Branch':
            return Get_Question(df,COL,n),'Branch',col_list  #para diferenciar Branch e Leaf
        else:
            print(col_list[tree_degree][2])
            raise Exception('Uknown col_list value, deve ser só Tree/Branch')
    col_size = len(col_list)
    count = 0 #* (ver comentário inicia, linha 10)
    found = False
    while not(found):
        if count > limit_count:
            col_list[-1][2]=['Leaf']
            return NB_Leaf(df),'LeafDone' #quando é TreeDone, é pq devolve um Tree e não uma pergunta
            
        r = random.randrange(col_size)
        COL = col_list[r]
        if COL not in picked:
            COL_unique = len(var_values(df,COL)) #qnt elementos tem a COL na df 
            n = Get_ColDiv(COL_unique)#nº de divisoes para fazer à df
            #agr vamos ver se esta divisão é boa ou não
            Q_done = False
            c_c = 0
            while (c_c<=col_count):
                df_n = len(df)//n
                if limit_up <= df_n:
                    col_list+=[[COL,n,'Branch']]
                    return Get_Question(df,COL,n),'Branch',col_list  #para diferenciar Branch e Leaf
                elif limit_down >= df_n:
                    n+=1
                    c_c+=1
                elif limit_up >= df_n >= limit_down:
                    col_list+=[[COL,n,'Leaf']]
                    return Get_Question(df,COL,n),'Leaf',col_list  #para diferenciar Branch e Leaf
                else:
                    raise Exception('df_n has unknown relation')
            count += 1
                

#Alterações a fazer:
#- Quando a coluna não funciona, tentar primeiro mudar o nº de divisões
#Se for impossível encontrar um número inteiro, aumentamos o count
#(Neste caso o limit_count vai ter de ser relativamente mais pequeno
#Só para o programa ficar mais rápido)

#- Ter mais um argumento -> em que "grau da árvore" vamos
#Assim, se estivermos no grau 2 e já tivermos a pergunta de grau 2 pq já foi encontrada
#Já não precisamos de encontrar uma coluna ao calhas
#"""
            

In [7]:
def Get_Col(df,colunas,picked): # devolve uma coluna aleatória
    done = False
    limit = max(int(len(colunas)/2),30)
    count = 0
    while not(done):
        if count >= limit:
            if len(picked)>=((np.size(df,1)-1)*0.6): #60% das colunas já escolidas
                return 'RLeaf'
            else:
                print(len(df))
                print('Wow, muitas colunas com apenas 1 valor\nOu a calhar sempre no picked\nSe calhar algo está mal')
                print('picked: ',sorted(picked))
                print('columns: ',np.size(df,1))
        r = random.randrange(len(colunas))
        COL = colunas[r]
        if len(var_values(df,COL))>1 and (COL not in picked): #se =1, não dá para fazer perguntas como é óbvio
            return COL
        count+=1

In [8]:
def Col_Exists(df,COL,spec,limit_down,limit_up): #Devolve instrução
    #Serve para quando estamos num grau da árvore que já foi analisado
    #Logo já tem uma pergunta guardada
    #spec -> indica se a coluna deverá representar uma divisão que dá para:
    #ainda outro branch('Branch') ou uma Leaf ('Leaf')
    if limit_down >= limit_up:
        raise Exception('Trocaste o down com o up') #tirar isto se o prg funcionar
    
    unique = len(var_values(df,COL))
    if spec == 'Leaf':
        if unique==1: 
            return 0,'RLeaf'  #df vai virar Leaf
        n,new_spec = Get_ColDiv_Adv(df,unique,limit_down,limit_up,mode='Leaf') #devolve o nº de divisões a fazer
        return n,new_spec
    
    elif spec == 'Branch':
        if unique==1:
            return 0,'RLeaf' #df vai virar Leaf
        n,new_spec = Get_ColDiv_Adv(df,unique,limit_down,limit_up,mode='Branch') #devolve o nº de divisões a fazer
        return n,new_spec
        
    else:
        print(spec)
        raise Exception('Unknown spec value')

In [9]:
def Get_ColDiv_Adv(df,n,limit_down,limit_up,mode='Branch'): 
    if limit_down >= limit_up:
        raise Exception('Trocaste o down com o up') #tirar isto se o prg funcionar
    #uma versão avançada do Get_ColDiv
    #agr tem em conta os limites que estabelemos
    n = Get_ColDiv(n)
    done = False
    Leafwarning = ''
    #serve para evitar casos onde é impossível uma suposta Leaf calhar no delta(Leaf)
    #oscilando entre valores deamsiado pequenos e grandes
    while not(done):
        df_n = len(df)//n
        
        if mode=='Branch':
            if df_n>limit_up:
                return n,mode
            elif limit_up<=df_n<=limit_up:
                if n==2:
                    #print('içi1')
                    return n,'Leaf'
                else:
                    n-=1
            else:
                if n==2:
                    return n,'RLeaf'
                else:
                    n-=1
        
        elif mode=='Leaf':
            if limit_up<=df_n<=limit_up:
                #print('içi3')
                return n,mode
            elif df_n>limit_up:
                if n==5 or ('ab' in Leafwarning): #se houber 'ab', anda a saltitar entre este elif e o else num ciclo inf 
                    return n,'Branch'
                else:
                    Leafwarning += 'a'
                    n+=1
            else:
                if n==2:
                    return n,'RLeaf'
                else:
                    Leafwarning += 'b'
                    n-=1
        
        elif mode == 'Simple': #quando ainda não temos pergunta para este grau
            if df_n>limit_up:
                return n,'Branch'
            elif limit_down<=df_n<=limit_up:
                #print('içi2',n)
                return n,'Leaf'
            else:
                if n==2:
                    return n,'RLeaf'
                else:
                    n-=1
        else:
            print(mode,'mode')
            raise Exception('Unknown mode value')

In [10]:
def Col_DNE(df,colunas,picked,limit_down,limit_up):
    if limit_down >= limit_up:
        raise Exception('Trocaste o down com o up') #tirar isto se o prg funcionar
    COL = Get_Col(df,colunas,picked)
    if COL == 'RLeaf':
        return 'Lixo1','Lixo2',COL
    unique = len(var_values(df,COL))
    
    n,new_spec = Get_ColDiv_Adv(df,unique,limit_down,limit_up,mode='Simple')
    return COL,n,new_spec

In [11]:
def Column_Pick(df,colunas,col_list,tree_degree,picked,limit_down,limit_up):
    if limit_down >= limit_up:
        raise Exception('Trocaste o down com o up') #tirar isto se o prg funcionar
    if tree_degree+1 <= len(col_list): #coluna existe
        COL = col_list[tree_degree][0]
        spec = col_list[tree_degree][1]
        
        #print(Col_Exists(df,COL,spec,limit_down,limit_up))
        n,new_spec = Col_Exists(df,COL,spec,limit_down,limit_up)

        if new_spec=='RLeaf':
            return 'Lixo',new_spec,col_list
        else:
            return Get_Question(df,COL,n),new_spec,col_list
    else: #coluna não existe
        COL,n,spec=Col_DNE(df,colunas,picked,limit_down,limit_up)
        if spec=='RLeaf':
            return 'Lixo',spec,col_list
        else:
            picked += [COL]
            col_list+=[[COL,spec]]
            return Get_Question(df,COL,n),spec,col_list
        

In [12]:
def build_NBTree(df,colunas,col_list,tree_degree,picked,limit_down,limit_up,spec):
    #print(len(df))
    if spec=='Leaf':
        #print('Leaf',len(df))
        return NB_Leaf(df),picked 
    
    Q,spec,col_list = Column_Pick(df,colunas,col_list,tree_degree,picked,limit_down,limit_up)
    
    if spec=='RLeaf':
        #print('RLeaf',len(df))
        return NB_Leaf(df),picked
    
    tree_degree+=1
    branches = NB_partition(df,Q)
    Branches = []
    Pickz = []
    #print('tree_degree',tree_degree)
    for i in range(len(branches)):
        if len(branches[i])!=0:
            ok,picked1 = build_NBTree(branches[i],colunas,col_list,tree_degree,picked,limit_down,limit_up,spec)
        Branches += [ok]
        Pickz += [picked]
        PICK = Pickz[np.argmax(list(map(lambda x:len(x),Pickz)))]
    return NB_Decision_Node(Q,Branches),PICK

In [19]:
np.argmax([2,3,5,3,2,4,6,2,1])

6

In [20]:
def build_NBForest(df,trees,Picked=[],Min=6,Max=8,Make_Tdf=False):
    F = NB_Forest()
    the_df=df.values
    if Make_Tdf:
        the_df,T_df=adjust_column_values(the_df,mini=Min,maxi=Max,perguntas=2)
    else:
        T_df='None'
    Picked=[]
    for t in range(trees):
        if t%250==0:
            print(t, 'in',trees)
        t,piky = build_NBTree(the_df,list(range(len(df.columns)-1)),[],0,[],10,25,None) 
        #10-limit_down/25->limit_up, podemos alterar
        if piky not in Picked: #não queremos árvores iguais
            Picked += [piky]
            F.add_tree(t)
    return F,T_df

In [21]:
#%%timeit
tree_tit,pp = build_NBTree(df_tit.values,list(range(len(df_tit.columns)-1)),[],0,[],10,25,None)

NameError: name 'df_tit' is not defined

In [29]:
F,tt=build_NBForest(df_tit,100)

714
164
164
154
154
118
25
24
24
24
21
36
(2, 'RLeaf')
10
(0, 'RLeaf')
137
(2, 'Branch')
137
(2, 'Branch')
133
(2, 'Branch')
133
(2, 'Branch')
99
(3, 'Branch')
34
30
4
33
(2, 'RLeaf')
32
(2, 'RLeaf')
34
(2, 'RLeaf')
4
(0, 'RLeaf')
145
(2, 'Branch')
145
(2, 'Branch')
139
(2, 'Branch')
139
(2, 'Branch')
101
(4, 'Leaf')
26
25
25
25
38
(2, 'RLeaf')
6
(2, 'RLeaf')
139
(2, 'Branch')
139
(2, 'Branch')
134
(2, 'Branch')
85
(2, 'Branch')
54
(2, 'Branch')
28
(2, 'RLeaf')
26
(2, 'RLeaf')
31
(2, 'RLeaf')
49
(2, 'RLeaf')
5
(2, 'RLeaf')
129
(2, 'Branch')
129
(2, 'Branch')
124
(2, 'Branch')
69
(2, 'Branch')
43
(2, 'RLeaf')
26
(2, 'RLeaf')
55
(2, 'Branch')
43
(2, 'RLeaf')
12
(2, 'RLeaf')
5
(2, 'RLeaf')
714
359
257
52
16
11
13
12
51
(2, 'Leaf')
29
22
51
(2, 'Leaf')
27
24
51
(2, 'Leaf')
28
23
52
(2, 'Branch')
29
20
9
23
(2, 'RLeaf')
102
(3, 'Branch')
35
(2, 'RLeaf')
34
(2, 'RLeaf')
33
(2, 'RLeaf')
355
(2, 'Branch')
264
(5, 'Branch')
54
(2, 'Branch')
29
(2, 'RLeaf')
25
(2, 'RLeaf')
53
(2, 'Branch')
29
(2

20
20
20
19
8
(2, 'RLeaf')
109
(2, 'Branch')
109
(2, 'Branch')
109
(2, 'Branch')
107
(4, 'Branch')
28
22
6
27
(2, 'RLeaf')
27
(2, 'RLeaf')
25
(2, 'RLeaf')
2
(2, 'RLeaf')
97
(2, 'Branch')
97
(2, 'Branch')
97
(2, 'Branch')
92
(3, 'Branch')
32
(2, 'RLeaf')
31
(2, 'RLeaf')
29
(2, 'RLeaf')
5
(2, 'RLeaf')
106
(2, 'Branch')
106
(2, 'Branch')
68
(2, 'Branch')
66
(2, 'Branch')
34
(2, 'RLeaf')
32
(2, 'RLeaf')
2
(2, 'RLeaf')
38
(2, 'RLeaf')
101
(2, 'Branch')
101
(2, 'Branch')
54
(2, 'Branch')
52
(2, 'Branch')
27
(2, 'RLeaf')
25
(2, 'RLeaf')
2
(2, 'RLeaf')
47
(2, 'RLeaf')
193
(5, 'Branch')
40
(2, 'RLeaf')
41
(2, 'RLeaf')
39
(2, 'RLeaf')
40
(2, 'RLeaf')
33
(2, 'RLeaf')
714
164
122
112
112
112
23
22
22
22
23
10
(0, 'RLeaf')
42
(2, 'RLeaf')
137
(2, 'Branch')
114
(2, 'Branch')
110
(2, 'Branch')
110
(2, 'Branch')
110
(4, 'Branch')
29
24
5
28
(2, 'RLeaf')
28
(2, 'RLeaf')
25
(2, 'RLeaf')
4
(0, 'RLeaf')
23
(2, 'RLeaf')
145
(2, 'Branch')
128
(2, 'Branch')
123
(2, 'Branch')
123
(2, 'Branch')
123
(4, 'Branch

108
85
18
17
17
17
16
23
(2, 'RLeaf')
109
(2, 'Branch')
86
(3, 'Branch')
30
30
29
(2, 'RLeaf')
27
(2, 'RLeaf')
23
(2, 'RLeaf')
97
(2, 'Branch')
76
(3, 'Leaf')
26
25
25
21
(2, 'RLeaf')
106
(2, 'Branch')
79
(3, 'Branch')
27
(2, 'RLeaf')
26
(2, 'RLeaf')
26
(2, 'RLeaf')
27
(2, 'RLeaf')
101
(2, 'Branch')
78
(3, 'Branch')
27
(2, 'RLeaf')
26
(2, 'RLeaf')
25
(2, 'RLeaf')
23
(2, 'RLeaf')
193
(5, 'Branch')
40
(2, 'RLeaf')
41
(2, 'RLeaf')
39
(2, 'RLeaf')
40
(2, 'RLeaf')
33
(2, 'RLeaf')
714
684
154
32
17
15
31
(2, 'RLeaf')
31
(2, 'RLeaf')
31
(2, 'RLeaf')
29
(2, 'RLeaf')
133
(5, 'Branch')
28
(2, 'RLeaf')
27
(2, 'RLeaf')
27
(2, 'RLeaf')
27
(2, 'RLeaf')
24
(2, 'RLeaf')
139
(5, 'Branch')
29
(2, 'RLeaf')
28
(2, 'RLeaf')
28
(2, 'RLeaf')
28
(2, 'RLeaf')
26
(2, 'RLeaf')
134
(5, 'Branch')
28
(2, 'RLeaf')
27
(2, 'RLeaf')
27
(2, 'RLeaf')
27
(2, 'RLeaf')
25
(2, 'RLeaf')
124
(4, 'Branch')
32
(2, 'RLeaf')
31
(2, 'RLeaf')
31
(2, 'RLeaf')
30
(2, 'RLeaf')
30
(2, 'RLeaf')
714
144
30
19
11
29
(2, 'RLeaf')
31
(2, 'RL

24
34
(2, 'RLeaf')
145
(2, 'Branch')
145
(2, 'Branch')
106
(4, 'Branch')
27
27
26
(2, 'RLeaf')
26
(2, 'RLeaf')
27
(2, 'RLeaf')
39
(2, 'RLeaf')
139
(2, 'Branch')
139
(2, 'Branch')
92
(3, 'Branch')
32
(2, 'RLeaf')
31
(2, 'RLeaf')
29
(2, 'RLeaf')
47
(2, 'RLeaf')
129
(2, 'Branch')
129
(2, 'Branch')
90
(3, 'Branch')
31
(2, 'RLeaf')
30
(2, 'RLeaf')
29
(2, 'RLeaf')
39
(2, 'RLeaf')
714
521
404
383
383
383
97
20
19
19
19
20
64
(2, 'Branch')
33
31
71
(2, 'Branch')
37
34
77
(3, 'Leaf')
27
26
24
74
(2, 'Branch')
38
36
21
(2, 'RLeaf')
117
(2, 'Branch')
115
(2, 'Branch')
76
(2, 'Branch')
76
(2, 'Branch')
40
(2, 'RLeaf')
36
(2, 'RLeaf')
39
(2, 'RLeaf')
2
(0, 'RLeaf')
193
(2, 'Branch')
148
(2, 'Branch')
145
(2, 'Branch')
92
(2, 'Branch')
53
(2, 'Branch')
27
(2, 'RLeaf')
26
(2, 'RLeaf')
39
(2, 'RLeaf')
53
(2, 'Branch')
30
(2, 'RLeaf')
23
(2, 'RLeaf')
3
(0, 'RLeaf')
45
(2, 'RLeaf')
714
714
164
34
22
12
33
(2, 'RLeaf')
33
(2, 'RLeaf')
33
(2, 'RLeaf')
31
(2, 'RLeaf')
137
(5, 'Branch')
28
(2, 'RLeaf')
27
(

(3, 'Branch')
27
(2, 'RLeaf')
26
(2, 'RLeaf')
25
(2, 'RLeaf')
59
(2, 'Branch')
31
(2, 'RLeaf')
28
(2, 'RLeaf')
106
(2, 'Branch')
76
(2, 'Branch')
39
(2, 'RLeaf')
37
(2, 'RLeaf')
30
(2, 'RLeaf')
714
359
257
179
46
46
50
(2, 'Leaf')
50
42
(2, 'RLeaf')
41
(2, 'RLeaf')
78
(3, 'Branch')
28
(2, 'RLeaf')
28
(2, 'RLeaf')
22
(2, 'RLeaf')
102
(2, 'Branch')
92
(3, 'Branch')
35
(2, 'RLeaf')
28
(2, 'RLeaf')
29
(2, 'RLeaf')
10
(2, 'RLeaf')
355
(2, 'Branch')
264
(2, 'Branch')
225
(4, 'Branch')
68
(2, 'Branch')
68
15
14
14
14
11
52
(2, 'Branch')
52
(2, 'Branch')
27
26
1
25
(2, 'RLeaf')
49
(2, 'RLeaf')
56
(2, 'Branch')
56
(2, 'Branch')
29
(2, 'RLeaf')
27
(2, 'RLeaf')
39
(2, 'RLeaf')
91
(2, 'Branch')
56
(2, 'Branch')
30
(2, 'RLeaf')
26
(2, 'RLeaf')
35
(2, 'RLeaf')
714
359
222
218
175
175
36
13
12
11
35
(2, 'RLeaf')
35
(2, 'RLeaf')
35
(2, 'RLeaf')
34
(2, 'RLeaf')
43
(2, 'RLeaf')
4
(0, 'RLeaf')
137
(2, 'Branch')
135
(2, 'Branch')
76
(2, 'Branch')
76
(2, 'Branch')
39
(2, 'RLeaf')
37
(2, 'RLeaf')
59
(2, 'Br

(2, 'Branch')
36
(2, 'RLeaf')
32
(2, 'RLeaf')
6
(2, 'RLeaf')
355
(2, 'Branch')
331
(5, 'Branch')
67
(2, 'Branch')
36
(2, 'RLeaf')
31
(2, 'RLeaf')
66
(2, 'Branch')
35
(2, 'RLeaf')
31
(2, 'RLeaf')
66
(2, 'Branch')
34
(2, 'RLeaf')
32
(2, 'RLeaf')
66
(2, 'Branch')
35
(2, 'RLeaf')
31
(2, 'RLeaf')
66
(2, 'Branch')
37
(2, 'RLeaf')
29
(2, 'RLeaf')
24
(2, 'RLeaf')
714
359
359
80
80
17
16
16
16
15
77
(2, 'Branch')
76
(3, 'Leaf')
26
25
25
1
(0, 'RLeaf')
64
(2, 'Branch')
63
(2, 'Branch')
33
27
6
30
(2, 'RLeaf')
1
(0, 'RLeaf')
70
(2, 'Branch')
68
(2, 'Branch')
35
(2, 'RLeaf')
33
(2, 'RLeaf')
2
(2, 'RLeaf')
68
(2, 'Branch')
66
(2, 'Branch')
34
(2, 'RLeaf')
32
(2, 'RLeaf')
2
(2, 'RLeaf')
355
(2, 'Branch')
355
(5, 'Branch')
78
(2, 'Branch')
71
(2, 'Branch')
37
(2, 'RLeaf')
34
(2, 'RLeaf')
7
(2, 'RLeaf')
83
(2, 'Branch')
77
(3, 'Leaf')
27
26
24
6
(2, 'RLeaf')
55
(2, 'Branch')
54
(2, 'Branch')
28
(2, 'RLeaf')
26
(2, 'RLeaf')
1
(0, 'RLeaf')
69
(2, 'Branch')
64
(2, 'Branch')
33
(2, 'RLeaf')
31
(2, 'RLeaf'

95
95
27
22
24
22
94
(2, 'Branch')
49
(2, 'RLeaf')
45
(2, 'RLeaf')
94
(2, 'Branch')
94
(3, 'Branch')
33
25
8
30
(2, 'RLeaf')
31
(2, 'RLeaf')
94
(2, 'Branch')
48
(2, 'RLeaf')
46
(2, 'RLeaf')
94
(2, 'Branch')
94
(3, 'Branch')
33
(2, 'RLeaf')
30
(2, 'RLeaf')
31
(2, 'RLeaf')
243
(5, 'Branch')
50
(2, 'Leaf')
50
49
(2, 'RLeaf')
49
(2, 'RLeaf')
49
(2, 'RLeaf')
46
(2, 'RLeaf')
714
714
164
164
122
25
24
24
24
25
42
(2, 'RLeaf')
137
(2, 'Branch')
137
(2, 'Branch')
114
(4, 'Branch')
29
24
5
28
(2, 'RLeaf')
28
(2, 'RLeaf')
29
(2, 'RLeaf')
23
(2, 'RLeaf')
145
(2, 'Branch')
145
(2, 'Branch')
128
(5, 'Leaf')
27
26
26
26
23
17
(2, 'RLeaf')
139
(2, 'Branch')
87
(2, 'Branch')
68
(2, 'Branch')
35
(2, 'RLeaf')
33
(2, 'RLeaf')
19
(2, 'RLeaf')
52
(2, 'Branch')
38
(2, 'RLeaf')
14
(2, 'RLeaf')
129
(2, 'Branch')
71
(2, 'Branch')
54
(2, 'Branch')
28
(2, 'RLeaf')
26
(2, 'RLeaf')
17
(2, 'RLeaf')
58
(2, 'Branch')
47
(2, 'RLeaf')
11
(2, 'RLeaf')
714
144
136
99
99
31
20
26
22
37
(2, 'RLeaf')
8
(2, 'RLeaf')
143
(2, '

In [30]:
F.size()

97

In [14]:
def remove_columns(file):
    file=file.drop(['League','Date','Time','HT','AT'],axis=1)
    try:
        file=file.drop(['ODDH_Aver.','ODDD_Aver.','ODDA_Aver.'],axis=1)
    except:
        pass
    return file

def var_values(df,column): #dá nos uma lista dos valores de uma variável 
    #Nota, cada variável está associada a uma coluna
    return np.unique(df[:,column])

def depend_counts(df):
    #devolve um dicionário com os valores da variável dependente e as vezes q aparecem
    #NOTA, esta função assume q a variável depend está na última coluna da df
    coluna=df[:,-1] #que neste caso é np.array
    varz,counts=np.unique(coluna,return_counts=True)
    if len(counts)==2:
        return {0:counts[0],1:counts[1]} #lista com dois elementos
    else:
        if varz[0]==0:
            return {0:counts[0],1:0}
        else:
            return {0:0,1:counts[0]}
        
def proba(df):
    #prob1=time.perf_counter()
    #calcula a probabilidade dos valores de um dicionário (em percentagem, para evitar float errors)
    #É necessário para as Leafs
    ind=len(df)
    dicti=depend_counts(df)
    new={0:round(100*(dicti[0]/ind),1),1:round(100*(dicti[1]/ind),1)}  # este módulo de3 decision trees está feito para var dep de 0/1 APENAS
    #prob2=time.perf_counter()
    #print(f'proba in {round(prob2-prob1,6)} seconds')
    return new

def adjust_column_values(df2,mini=5,maxi=9,perguntas=2):
    #ajusta os valores de cada coluna se for para fazer um modelo 
    #usando os dois tipos de pergunta (perguntas=2)
    #mini, maxi são o min e máx possíveis do número de valores de uma coluna
    #comparando com o número de registos (linhas)
    df=np.round(df2,1)
    if perguntas==2:
        Max=round((len(df)*maxi)/100)
        Min=round((len(df)*mini)/100)
        print('Max len: ',Max)
        print('Min len: ',Min)
        tens=[]
        acv_coefs=[]
        for i in range(len(df[0])): #df é uma np.array
            acv_coef='None'
            c_val,ten=check_float(df[:,i])
            unique=np.unique(c_val)
            if len(unique)>Max:
                new,acv_coef=aux_acv(c_val,Max,Min)
                df[:,i]=new
            tens+=[ten]
            acv_coefs+=[acv_coef]
        t_df = pd.DataFrame({'Check_Float':tens,'ACV_Coef':acv_coefs})
        return df,t_df
    elif perguntas==1:
        return df,'None'
    else:
        print('Só existem perguntas 1/2')
        raise('Erro')

def aux_acv(c_val,Max,Min,div=10,st_div=10,last='max'): 
    #div é o número que usamos para a divisão inteira
    #last é se no último passo divemos de reduzir ou aumentar (decidi começar em max só pq sim)
    #st_div são os passos que vamos dando para encontrar o div certo
    div = round(div,1) #caso contrário temos aqueles números que nunca mais acabam
    new_val2=np.array(list(map(lambda x:x//div,c_val)))
    new_val=np.unique(new_val2)
    #print(div,'diiiiiiiiiiiiiiiiiv')
    #print('len: ',len(new_val))
    if len(new_val)>Max:
        if last=='max':
            return aux_acv(c_val,Max,Min,div=div+st_div,st_div=st_div,last='max')
        else:
            return aux_acv(c_val,Max,Min,div=div+st_div,st_div=max(0.1,st_div//2),last='max')
    elif len(new_val)<=Min:
        if last=='min':
            return aux_acv(c_val,Max,Min,div=max(1,div-(max(0.1,st_div//2))),st_div=st_div,last='min')
        else:
            return aux_acv(c_val,Max,Min,div=max(1,div-(max(0.1,st_div//4))),st_div=max(0.1,st_div//2),last='min')
    else:
        return new_val2,div
    
def check_float(c_val,ten=0):
    #serve para tornar colunas com valores decimais em valores inteiros
    tf_val=np.array(list(map(lambda x:(x).is_integer(),c_val)))
    if ten==200:
        print(c_val)
    if all(tf_val):
        return c_val,ten
    else:
        ten+=1
        n_val=np.array(list(map(lambda x:x*10,c_val)))
        return check_float(n_val,ten)

[1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, 76]

In [129]:
100000*0.61

61000.0

In [130]:
61000//3600

16

In [142]:
len(df)

3000

In [16]:
def get_leaf_beta(tree):
    if isinstance(tree,NB_Leaf):
        print(tree.predictions)
        return tree.predictions[0]+tree.predictions[1]
    soma = 0
    for i in range(tree.branches_size):
        soma += get_leaf_beta(tree.branches[i])
    return soma

In [117]:
get_leaf_beta(tree_tit)

{0: 18, 1: 4}
{0: 14, 1: 6}
{0: 13, 1: 6}
{0: 12, 1: 5}
{0: 19, 1: 8}
{0: 17, 1: 12}
{0: 14, 1: 7}
{0: 19, 1: 8}
{0: 15, 1: 11}
{0: 16, 1: 8}
{0: 21, 1: 6}
{0: 12, 1: 14}
{0: 18, 1: 6}
{0: 29, 1: 11}
{0: 29, 1: 11}
{0: 25, 1: 9}
{0: 18, 1: 4}
{0: 14, 1: 6}
{0: 13, 1: 6}
{0: 12, 1: 5}
{0: 19, 1: 8}
{0: 17, 1: 12}
{0: 14, 1: 7}
{0: 19, 1: 8}
{0: 15, 1: 11}
{0: 16, 1: 8}
{0: 21, 1: 6}
{0: 12, 1: 14}
{0: 18, 1: 6}
{0: 29, 1: 11}
{0: 29, 1: 11}
{0: 25, 1: 9}
{0: 7, 1: 36}
{0: 14, 1: 9}
{0: 13, 1: 9}
{0: 25, 1: 12}
{0: 17, 1: 19}
{0: 16, 1: 20}
{0: 15, 1: 21}
{0: 17, 1: 20}
{0: 25, 1: 12}
{0: 17, 1: 19}
{0: 16, 1: 20}
{0: 15, 1: 21}
{0: 17, 1: 20}
{0: 15, 1: 17}
{0: 16, 1: 5}
{0: 7, 1: 1}


1359

In [17]:
df_tit = pd.read_csv('titanic_train.csv')

In [18]:
surv = df_tit['Survived']

In [19]:
df_tit=df_tit.drop(['Name','Ticket','Fare','Cabin','Survived'],axis=1)
df_tit['Survived']=surv

In [20]:
df_tit['Embarked']=[1 if i=='S' else 0 if i=='C' else 2 for i in list(df_tit['Embarked'])]

In [21]:
df_tit=df_tit.dropna(axis=0,how='any')

In [100]:
np.unique(list(df_tit['Embarked']))

array(['C', 'Q', 'S', 'nan'], dtype='<U3')

# Introducing new predict metod
- Aquela ideia de ponderar vários intervalos, não apenas um
- Este vai ser para NB_Forests, dps temos de adaptar aos outros

In [19]:
index=np.array([[inx] for inx in range(len(df_tit))]) #criamos a coluna index
df_tit2 = np.append(df_tit,index,axis=1)

In [15]:
def classify_dfNB(df,node,var=1):
    #não esquecer que a coluna -1 é o índice
    if isinstance(node,NB_Leaf):
        prob=node.probability
        return [[prob[var],list(df[:,-1])]] #-1 é o 'Index'  
    Parts=NB_partition(df,node.question)
    lix = []
    for i in range(node.branches_size):
        lix += classify_dfNB(Parts[i],node.branches[i])
    return lix



In [23]:
aa=classify_dfNB(df_tit2,tree_tit)

NameError: name 'df_tit2' is not defined

In [16]:
def prob_to_listNB(df,ko):
    out = [0]*len(df)
    for i in range(len(ko)):
        for n in ko[i][1]:
            out[int(n)]=ko[i][0]
    return np.array([out])



#ko2=np.array([list(reduce(prob_to_list,ko,[0]*len(df)))])

In [222]:
#%%timeit
np.array([list(reduce(prob_to_list,aa,[0]*len(df_tit2)))])

array([[23.7, 55.6, 29.4, 55.6, 29.4, 27.8, 23.7, 29.4, 60. , 23.7, 27.8,
        23.7, 29.4, 23.7, 27.8, 23.7, 29.4, 55.6, 55.6, 23.7, 35.3, 23.7,
        29.4, 60. , 55.6, 27.8, 35.3, 55.6, 23.7, 23.7, 23.7, 29.4, 35.3,
        60. , 23.7, 23.7, 23.7, 23.7, 27.8, 35.3, 27.8, 60. , 29.4, 60. ,
        23.7, 23.7, 55.6, 27.8, 23.7, 35.3, 23.7, 23.7, 29.4, 35.3, 23.7,
        60. , 29.4, 29.4, 29.4, 60. , 29.4, 23.7, 29.4, 35.3, 60. , 29.4,
        23.7, 60. , 29.4, 29.4, 23.7, 27.8, 29.4, 29.4, 27.8, 60. , 55.6,
        55.6, 29.4, 60. , 29.4, 29.4, 29.4, 23.7, 29.4, 27.8, 23.7, 23.7,
        23.7, 23.7, 23.7, 29.4, 35.3, 35.3, 23.7, 60. , 55.6, 55.6, 27.8,
        23.7, 29.4, 29.4, 29.4, 23.7, 29.4, 35.3, 35.3, 60. , 60. , 55.6,
        23.7, 35.3, 23.7, 29.4, 23.7, 60. , 60. , 29.4, 23.7, 55.6, 55.6,
        27.8, 60. , 29.4, 29.4, 27.8, 23.7, 29.4, 23.5, 55.6, 23.5, 37.8,
        37.8, 37.8, 23.5, 23.5, 27.8, 37.8, 37.8, 37.8, 27.8, 37.8, 27.8,
        35.3, 23.5, 37.8, 60. , 37.8, 

In [224]:
#%%timeit
a1=prob_to_list2(df_tit2,aa)
a2=np.array([list(reduce(prob_to_list,aa,[0]*len(df_tit2)))])
tp2=np.concatenate((a1,a2))

In [227]:
tp2.T[0][:]

array([23.7, 23.7])

In [17]:
def tree_probaNB(df,forest):
    size = forest.size()
    for t in range(size):
        node = forest.get_tree(t)
        ko = classify_dfNB(df,node)
        ko2 = prob_to_listNB(df,ko)
        if t==0:
            tp2=np.array(ko2)
        else:
            tp2=np.concatenate((tp2,ko2))                
    return tp2.T #=proba_list

In [41]:
proba_list = tree_probaNB(df_tit2,F)
ER=Extract_Results(proba_list)

[52.9 31.  16.7 30.8 33.3 23.3 28.6 33.3 52.9 38.3 25.8 52.9 28.6 28.
 28.  33.3 40.  34.4 40.  23.3 43.8 43.8 30.  28.6 28.6 34.4 43.8 32.
  3.7 24.  39.1 28.6 20.7 48.4 24.1 34.4 40.  32.7 28.6 40.7 40.  52.9
 30.8 21.7 52.9 23.1 28.6 21.7  4.  33.3 23.1 21.7 28.6 33.3 26.5 23.1
 25.8 25.8 34.6 26.5 25.8 28.2 23.1 30.6 25.  34.4 20.7 28.6 23.5 34.4
 25.8 25.  17.6 21.7 39.1 22.2 18.5 33.3 52.9 32.7 48.  28.  16.2 48.4
 20.7 28.1 23.1 34.4 34.4 23.1 32.  40.  25.8 20.  34.4 22.2  3.7]


In [18]:
def Extract_Results(proba_list,delta=5): #recebe output da tree_probaNB
    lix = list(range(0,100,delta))
    for r in range(len(proba_list)):
        row = proba_list[r]
        add_df = []
        v=0
        while v<(len(lix)-1):
            add_df += [len(np.where(np.logical_and(row>=lix[v],
                                           row<lix[v+1]))[0])]
            v+=1
        add_df+=[len(np.where(row>=lix[v])[0])]
        add_df += [sum(add_df)]
        add_df = np.array([add_df])
        if r==0:
            DF_array = add_df
        else:
            DF_array=np.concatenate((DF_array,add_df)) 
    C_list=[]
    for c in range(len(lix)):
        if c==(len(lix)-1):
            C_list+=[str(lix[c])+'__']
        else:
            C_list+=[str(lix[c])+'__'+str(lix[c+1])]
    C_list+=['Total']
    
    dfdf = pd.DataFrame(DF_array,columns=C_list)
    return dfdf

def EBI_aux(er_values,add,total):
    out=[]
    soma = []
    for s in range(len(add)):
        soma+=[sum(add[s])]
    out = list(map(lambda x: total-x, soma))
    return soma,out
        
def Extract_Big_intervals(er): #recebe o output do Extract Results
    new_c = []
    for c in list(er.columns)[:-2]: #não queremos a coluna Total
        new_c += ['__'+c.split('__')[1],c.split('__')[1]+'=__']
    total = er.iloc[0]['Total']   
    for c in range(len(list(er.columns)[:-2])):
        pre_add = er.values[:,:(c+1)]
        
        ADD1,ADD2 = EBI_aux(er.values,pre_add,total)
        ADD = np.array([ADD1]+[ADD2])
        if c==0:
            DF_array = ADD
        else:
            DF_array=np.concatenate((DF_array,ADD)) 
    new_c += ['Total']
    DF_array = np.concatenate((DF_array,np.array([[total]*len(ADD1)])))
    DF_array=DF_array.T
    dfdf = pd.DataFrame(DF_array,columns=new_c)
    return dfdf

def Convert_Perc(ER):
    total = ER.iloc[0]['Total']
    arr = []
    for i in range(len(ER)):
        new = list(map(lambda x:round(100*(x/total),2),list(ER.iloc[i][:])[:-1]))
        arr += [new]
    return pd.DataFrame(arr,columns=list(ER.columns)[:-1])

In [66]:
test_array = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
test_array[:,:1].T

array([[ 1,  4,  7, 10]])

In [126]:
Extract_Big_intervals(ER)

[[ 3 94  3 ... 97  0 97]
 [ 0 97  0 ... 96  1 97]
 [ 0 97  0 ... 97  0 97]
 ...
 [ 0 97  0 ... 97  0 97]
 [ 0 97  0 ... 97  0 97]
 [ 1 96  2 ... 97  0 97]] ['__5', '5=__', '__10', '10=__', '__15', '15=__', '__20', '20=__', '__25', '25=__', '__30', '30=__', '__35', '35=__', '__40', '40=__', '__45', '45=__', '__50', '50=__', '__55', '55=__', '__60', '60=__', '__65', '65=__', '__70', '70=__', '__75', '75=__', '__80', '80=__', '__85', '85=__', '__90', '90=__', '__95', '95=__', 'Total']


,__5,5=__,__10,10=__,__15,15=__,__20,20=__,__25,25=__,__30,30=__,__35,35=__,__40,40=__,__45,45=__,__50,50=__,__55,55=__,__60,60=__,__65,65=__,__70,70=__,__75,75=__,__80,80=__,__85,85=__,__90,90=__,__95,95=__,Total
0,3,94,3,94,3,94,7,90,28,69,52,45,76,21,79,18,88,9,91,6,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97
1,0,97,0,97,0,97,3,94,8,89,12,85,33,64,41,56,45,52,49,48,72,25,84,13,84,13,96,1,96,1,96,1,96,1,96,1,96,1,97
2,0,97,0,97,0,97,3,94,6,91,51,46,62,35,85,12,95,2,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97
3,0,97,0,97,0,97,0,97,0,97,4,93,25,72,34,63,45,52,47,50,70,27,86,11,86,11,96,1,96,1,96,1,96,1,96,1,96,1,97
4,7,90,13,84,18,79,23,74,26,71,32,65,39,58,62,35,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,15,82,17,80,18,79,25,72,42,55,49,48,62,35,63,34,70,27,73,24,81,16,81,16,81,16,88,9,89,8,93,4,97,0,97,0,97,0,97
710,0,97,0,97,0,97,0,97,1,96,23,74,40,57,57,40,69,28,88,9,94,3,96,1,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97
711,0,97,0,97,0,97,5,92,24,73,25,72,43,54,56,41,58,39,71,26,89,8,91,6,94,3,97,0,97,0,97,0,97,0,97,0,97,0,97
712,0,97,0,97,0,97,0,97,2,95,29,68,46,51,63,34,75,22,90,7,94,3,96,1,97,0,97,0,97,0,97,0,97,0,97,0,97,0,97


In [103]:
ER

,0__5,5__10,10__15,15__20,20__25,25__30,30__35,35__40,40__45,45__50,50__55,55__60,60__65,65__70,70__75,75__80,80__85,85__90,90__95,95__,Total
0,3,0,0,4,21,24,24,3,9,3,6,0,0,0,0,0,0,0,0,0,97
1,0,0,0,3,5,4,21,8,4,4,23,12,0,12,0,0,0,0,0,1,97
2,0,0,0,3,3,45,11,23,10,2,0,0,0,0,0,0,0,0,0,0,97
3,0,0,0,0,0,4,21,9,11,2,23,16,0,10,0,0,0,0,0,1,97
4,7,6,5,5,3,6,7,23,35,0,0,0,0,0,0,0,0,0,0,0,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,15,2,1,7,17,7,13,1,7,3,8,0,0,7,1,4,4,0,0,0,97
710,0,0,0,0,1,22,17,17,12,19,6,2,1,0,0,0,0,0,0,0,97
711,0,0,0,5,19,1,18,13,2,13,18,2,3,3,0,0,0,0,0,0,97
712,0,0,0,0,2,27,17,17,12,15,4,2,1,0,0,0,0,0,0,0,97


In [273]:
[1,2][:2]

[1, 2]

In [330]:
test_er=ER.iloc[:][list(ER.columns)[:4]]

In [332]:
test_er.values[0]

array([2, 1, 0, 7])

In [19]:
df_Train = pd.read_excel('Husky6SA_Train.xlsx')
df_Test = pd.read_excel('Husky6SA_Test.xlsx')
df_Train_,df_Test_ = remove_columns(df_Train),remove_columns(df_Test)

In [21]:
FSA,tt=build_NBForest(df_Train_,10000)

0 in 10000
250 in 10000
500 in 10000
750 in 10000
1000 in 10000
1250 in 10000
1500 in 10000
1750 in 10000
2000 in 10000
2250 in 10000
2500 in 10000
2750 in 10000
3000 in 10000
3250 in 10000
3500 in 10000
3750 in 10000
4000 in 10000
4250 in 10000
4500 in 10000
4750 in 10000
5000 in 10000
5250 in 10000
5500 in 10000
5750 in 10000
6000 in 10000
6250 in 10000
6500 in 10000
6750 in 10000
7000 in 10000
7250 in 10000
7500 in 10000
7750 in 10000
8000 in 10000
8250 in 10000
8500 in 10000
8750 in 10000
9000 in 10000
9250 in 10000
9500 in 10000
9750 in 10000


In [25]:
index=np.array([[inx] for inx in range(len(df_Test_))]) #criamos a coluna index
df_Test2 = np.append(df_Test_,index,axis=1)

In [26]:
proba_list = tree_probaNB(df_Test2,FSA)
ER=Extract_Results(proba_list)
ERI=Extract_Big_intervals(ER)

In [32]:
def Negative(ER,TL):
    good_c=list(ER.columns)[:-1]
    done=False
    c=0
    while c<len(good_c) and not(done):
        if str(TL) in good_c[c]:
            done = True
        else:
            c+=1
    if not(done):
        raise Exception(str(TL)+' not found in columns')
    arr = []
    for i in range(len(ER)):
        value = 0
        low = 100
        found = False
        for c in range(0,c+1):
            if not(found) and ER.iloc[i][good_c[c]]>0:
                low = int(good_c[c].split('__')[1])
                found = True
            value += ER.iloc[i][good_c[c]]*(int(good_c[c].split('__')[0])-100)
        arr += [[value,low]]
    return pd.DataFrame(arr,columns=['Negative_TL'+str(TL),'Lowest_%'])
        
    
            

In [33]:
Negative(ER,60).sort_values(['Negative_TL60'],ascending=[False])

,Negative_TL60,Lowest_%
13,-14470,25
85,-19225,30
29,-19515,20
49,-19820,30
71,-22545,10
83,-24395,25
12,-25000,25
21,-25315,15
30,-26215,10
134,-27010,10


In [38]:
df_Test.loc[[13,85,29,49,71,83,12,21,30,134]]

,League,Date,Time,HT,AT,H_xPts_Total,H_xPts_H,A_xPts_Total,A_xPts_A,H_xPts_Total5,H_xPts_H5,A_xPts_Total5,A_xPts_A5,xPoints_Diff_Total,xPoints_Diff_H-A,xPts_Diff_Total5,xPts_Diff_H-A5,HT_HPoints_5,AT_APoints_5,HA_HAPointsDiff,HT_TotalPoints_5,AT_TotalPoints_5,HA_TotalPointsDiff,HxG_5,Last_HxG,Last_HxPTS,HPass%_5,Last_HPass%,HPPDA_5,Last_HPPDA,HDEEP_5,Last_HDEEP,HCCC_5,Last_HCCC,HxConcd_5,Last_HxConcd,AxG_5,Last_AxG,Last_AxPTS,APass%_5,Last_APass%,APPDA_5,Last_APPDA,ADEEP_5,Last_ADEEP,ACCC_5,Last_ACCC,AxConcd_5,Last_AxConcd,xG_Diff_5,xPTS_Diff_5,Pass%_Diff_5,PPDA_Diff_5,DEEP_Diff_5,CCC_Diff_5,xConcd_Diff_5,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,Last_H_S,HST_5,Last_H_ST,HScrd_5,Last_H_Scrd,HConcd_5,Last_H_Concd,AS_5,Last_A_S,AST_5,Last_A_ST,AScrd_5,Last_A_Scrd,AConcd_5,Last_A_Concd,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,WinCounterTotal_H,WinCounterTotal_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ConcdCounterTotal_H,ConcdCounterTotal_A,WinPercTotal_H,WinPercTotal_A,ScrdPercTotal_H,ScrdPercTotal_A,ConcdPercTotal_H,ConcdPercTotal_A,WinCounterH_H,WinCounterA_A,ScrdCounterH_H,ScrdCounterTotalA_A,ConcdCounterH_H,ConcdCounterA_A,WinPercH_H,WinPercA_A,ScrdPercH_H,ScrdPercTotalA_A,ConcdPercH_H,ConcdPercA_A,Lucky_H5,Lucky_A5,Lucky_Diff5,Unlucky_H5,Unlucky_A5,Unlucky_Diff5,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,DEEPForm_H5,DEEPForm_A5,DEEPForm_Diff5,PPDAForm_H5,PPDAForm_A5,PPDAForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5,DeForm_H5,DeForm_A5,DeForm_Diff5,AtkForm_H5,AtkForm_A5,AtkForm_Diff5,Form1_H5,Form1_A5,Form1_Diff5,Form2_H5,Form2_A5,Form2_Diff5,Form3_H5,Form3_A5,Form3_Diff5,NewForm_H5,NewForm_A5,NewForm_Diff5,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
13,Serie_A,2020-01-20,1945,Atalanta,Spal,2.0,2.3,0.9,0.7,12.9,12.3,4.1,4.1,1.1,1.6,8.8,8.2,2.2,0.8,1.4,2.2,0.8,1.4,2.4,1,1,84,1,8,0,8,0,2,0,0.6,1,1.3,0,1,80,1,11,1,4,0,1,0,2.2,0,1.1,1.8,4,-3,4,1,-1.6,3.2,7.5,-4.3,20,0,8,0,3,0,1,0,13,0,2,0,1,0,2,0,7,6,2,-1,-1,-2,5,-2,-1,-5,-5,-11,37,-11,-5,-37,2,-2,2,2,2,-1,0,0,0,0,0,0,-3.2,-1.1,-2.1,10.0,0.0,10.0,3,-3,6,1,0,1,0,-3,3,-2,3,-5,1,2,-1,77,-105,182,114,-15,129,5.4,1.5,3.9,1.6,1.0,0.6,5.7,0.7,5.0,15.2,3.4,11.8,0,1.19,7.07,13.82
85,Serie_A,2020-07-15,1830,AC Milan,Parma,1.7,1.7,1.0,0.7,9.4,10.4,4.7,3.3,0.7,1.0,4.7,7.1,2.2,1.1,1.1,2.2,1.1,1.1,1.9,0,0,87,1,11,1,7,0,2,0,1.2,1,1.2,1,1,82,1,15,0,5,0,1,0,2.0,0,0.7,1.0,5,-4,2,1,-0.8,3.5,5.3,-1.8,14,0,6,0,3,0,1,1,12,1,3,1,1,1,2,0,2,3,2,-1,-1,-5,5,5,-2,-5,-3,-16,44,22,-6,-25,2,-1,2,2,0,-1,0,0,0,0,0,0,1.1,-3.2,4.3,2.2,4.0,-1.8,3,-2,5,1,-1,2,3,-2,5,-5,3,-8,3,1,2,33,-84,117,87,-25,112,2.8,1.1,1.7,1.2,2.2,-1.0,2.6,0.5,2.1,11.2,5.1,6.1,1,1.44,4.66,7.06
29,Serie_A,2020-02-02,1400,Atalanta,Genoa,2.1,2.3,1.2,1.0,13.4,12.6,5.5,5.4,0.9,1.3,7.9,7.2,1.8,0.5,1.3,1.8,0.5,1.3,3.3,1,1,83,0,8,0,12,1,2,1,0.7,0,1.4,1,1,75,1,11,0,5,0,2,0,1.9,0,1.9,1.6,8,-3,7,0,-1.2,3.8,2.9,0.9,22,1,9,1,4,1,1,0,9,1,5,1,1,0,2,0,13,4,3,-1,1,-3,5,-1,1,1,5,-14,43,-5,5,5,-1,1,2,2,0,1,-100,0,500,0,0,0,-4.2,-1.8,-2.4,11.0,2.4,8.6,4,0,4,1,0,1,5,0,5,-2,-1,-1,1,-5,6,57,-15,72,186,7,179,7.9,1.5,6.4,1.3,1.5,-0.2,6.7,1.1,5.6,13.0,6.5,6.5,0,1.24,6.15,11.58
49,Serie_A,2020-02-16,1400,Juventus,Brescia,1.8,2.3,1.0,1.0,7.5,12.9,2.1,4.3,0.8,1.3,5.4,8.6,2.8,0.6,2.2,2.8,0.6,2.2,1.5,0,0,87,0,8,0,6,1,1,0,1.3,1,1.0,0,0,73,0,12,0,4,0,1,0,2.6,0,0.5,1.1,14,-4,2,0,-1.3,3.1,3.5,-0.4,13,1,5,0,2,0,1,1,10,0,4,1,1,0,2,0,3,1,1,-1,-1,-5,5,2,-1,-5,-4,-35,87,9,-4,-35,2,-2,2,2,2,-1,0,0,0,0,0,0,0.0,0.2,-0.2,0.0,0.0,0.0,1,-1,2,0,0,0,0,-4,4,-4,2,-6,6,-7,13,24,-106,130,34,-69,103,2.2,0.7,1.5,1.3,1.2,0.1,2.2,0.2,2.0,8.6,2.3,6.3,1,1.18,7.26,15.95
71,Serie_A,2020-07-08,2045,Atalanta,Sampdoria,2.1,2.3,1.3,1.0,10.3,10.6,4.8,5.3,0.8,1.3,5.5,5.3,2.8,1.3,1.5,2.8,1.3,1.5,2.0,1,1,87,1,9,0,11,1,3,0,0.9,0,1.3,0,1,76,1,15,1,4,0,1,1,1.9,0,0.7,1.1,11,-6,7,2,-1.0,3.6,2.7,0.9,17,1,7,0,2,0,1,0,13,0,5,0,2,1,1,0,4,2,0,0,5,2,5,5,2,1,27,7,60,30,7,3,2,2,2,2,2,1,400,100,1000,500,200,-100,4.7,3.4,1.3,0.0,0.0,0.0,3,-1,4,1,-1,2,5,-4,9,-4,4,-8,5,-6,11,76

In [39]:
Convert_Perc(ER)

,0__5,5__10,10__15,15__20,20__25,25__30,30__35,35__40,40__45,45__50,50__55,55__60,60__65,65__70,70__75,75__80,80__85,85__90,90__95,95__
0,0.00,0.00,0.00,0.00,0.00,2.40,6.59,10.48,7.49,18.41,24.10,13.47,8.08,2.69,3.44,2.40,0.00,0.30,0.15,0.00
1,1.95,0.45,2.10,2.84,4.49,7.93,8.98,22.60,19.46,11.83,10.18,3.59,3.29,0.30,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,1.20,3.44,12.28,19.61,12.43,10.18,17.81,7.78,6.59,5.09,3.59,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,2.84,3.59,5.99,11.68,13.17,18.56,23.65,12.87,4.19,2.40,0.30,0.60,0.15,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.60,0.90,0.00,2.10,5.84,9.58,17.22,10.63,14.82,16.77,12.57,8.08,0.45,0.45,0.00
5,0.00,0.00,0.00,0.00,0.00,1.05,2.25,3.29,8.08,16.47,28.29,11.38,13.92,8.68,3.44,2.25,0.90,0.00,0.00,0.00
6,0.00,0.00,0.00,0.75,1.05,4.94,15.57,12.72,20.81,10.33,14.82,9.28,4.64,3.59,0.00,0.00,0.00,1.05,0.45,0.00
7,0.00,0.00,0.30,0.00,2.25,6.44,9.58,16.02,17.51,11.83,17.81,9.28,4.94,3.89,0.15,0.00,0.00,0.00,0.00,0.00
8,0.00,0.45,1.35,1.95,4.64,11.38,15.72,13.47,13.92,9.88,8.23,6.44,2.99,3.14,3.29,2.25,0.90,0.00,0.00,0.00
9,0.00,0.00,0.00,0.45,1.50,1.95,8.38,14.67,17.81,21.86,17.51,6.74,5.69,1.50,0.90,0.60,0.30,0.15,0.00,0.00


In [65]:
ER.loc[list(np.where(np.logical_and(df_Test['ODDH_Aver.']>=2,df_Test['ODDH_Aver.']<2.2))[0])]

,0__5,5__10,10__15,15__20,20__25,25__30,30__35,35__40,40__45,45__50,50__55,55__60,60__65,65__70,70__75,75__80,80__85,85__90,90__95,95__,Total
1,11,4,11,15,28,52,70,144,130,92,71,22,19,2,0,0,0,0,0,0,671
45,0,3,1,18,40,39,111,99,143,99,63,37,9,5,0,4,0,0,0,0,671
51,0,0,0,2,8,52,110,107,169,98,69,34,16,6,0,0,0,0,0,0,671
56,0,0,0,0,4,17,24,103,150,106,151,63,38,4,7,0,4,0,0,0,671
65,0,0,0,7,20,35,61,70,97,103,160,76,22,14,0,6,0,0,0,0,671
75,0,0,4,20,41,58,77,38,82,104,95,46,47,32,24,3,0,0,0,0,671
78,0,0,6,32,58,84,130,144,146,42,24,5,0,0,0,0,0,0,0,0,671
81,0,1,12,29,44,71,122,95,125,62,77,12,21,0,0,0,0,0,0,0,671
91,7,1,39,13,83,95,135,93,129,36,30,5,5,0,0,0,0,0,0,0,671
124,0,0,0,0,9,6,4,29,91,72,109,79,90,61,67,49,0,5,0,0,671


In [64]:
df_Test.loc[list(np.where(np.logical_and(df_Test['ODDH_Aver.']>=2,df_Test['ODDH_Aver.']<2.2))[0])]

,League,Date,Time,HT,AT,H_xPts_Total,H_xPts_H,A_xPts_Total,A_xPts_A,H_xPts_Total5,H_xPts_H5,A_xPts_Total5,A_xPts_A5,xPoints_Diff_Total,xPoints_Diff_H-A,xPts_Diff_Total5,xPts_Diff_H-A5,HT_HPoints_5,AT_APoints_5,HA_HAPointsDiff,HT_TotalPoints_5,AT_TotalPoints_5,HA_TotalPointsDiff,HxG_5,Last_HxG,Last_HxPTS,HPass%_5,Last_HPass%,HPPDA_5,Last_HPPDA,HDEEP_5,Last_HDEEP,HCCC_5,Last_HCCC,HxConcd_5,Last_HxConcd,AxG_5,Last_AxG,Last_AxPTS,APass%_5,Last_APass%,APPDA_5,Last_APPDA,ADEEP_5,Last_ADEEP,ACCC_5,Last_ACCC,AxConcd_5,Last_AxConcd,xG_Diff_5,xPTS_Diff_5,Pass%_Diff_5,PPDA_Diff_5,DEEP_Diff_5,CCC_Diff_5,xConcd_Diff_5,SQuality_H5,SQuality_A5,SQuality_Diff5,HS_5,Last_H_S,HST_5,Last_H_ST,HScrd_5,Last_H_Scrd,HConcd_5,Last_H_Concd,AS_5,Last_A_S,AST_5,Last_A_ST,AScrd_5,Last_A_Scrd,AConcd_5,Last_A_Concd,S_Diff_H-A_5,ST_Diff_H-A_5,Scrd_Diff_H-A_5,Concd_Diff_H-A_5,WinCounterTotal_H,WinCounterTotal_A,ScrdCounterTotal_H,ScrdCounterTotal_A,ConcdCounterTotal_H,ConcdCounterTotal_A,WinPercTotal_H,WinPercTotal_A,ScrdPercTotal_H,ScrdPercTotal_A,ConcdPercTotal_H,ConcdPercTotal_A,WinCounterH_H,WinCounterA_A,ScrdCounterH_H,ScrdCounterTotalA_A,ConcdCounterH_H,ConcdCounterA_A,WinPercH_H,WinPercA_A,ScrdPercH_H,ScrdPercTotalA_A,ConcdPercH_H,ConcdPercA_A,Lucky_H5,Lucky_A5,Lucky_Diff5,Unlucky_H5,Unlucky_A5,Unlucky_Diff5,STForm_H5,STForm_A5,STForm_Diff5,CCCForm_H5,CCCForm_A5,CCCForm_Diff5,DEEPForm_H5,DEEPForm_A5,DEEPForm_Diff5,PPDAForm_H5,PPDAForm_A5,PPDAForm_Diff5,PassForm_H5,PassForm_A5,PassForm_Diff5,DeForm_H5,DeForm_A5,DeForm_Diff5,AtkForm_H5,AtkForm_A5,AtkForm_Diff5,Form1_H5,Form1_A5,Form1_Diff5,Form2_H5,Form2_A5,Form2_Diff5,Form3_H5,Form3_A5,Form3_Diff5,NewForm_H5,NewForm_A5,NewForm_Diff5,Won/NotWon,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.
1,Serie_A,2020-01-12,1700,Verona,Genoa,1.2,1.2,1.3,1.0,7.5,7.5,9.1,5.5,-0.1,0.2,-1.6,2.0,1.4,0.6,0.8,1.4,0.6,0.8,1.9,1,1,73,1,8,1,5,1,3,1,1.7,0,1.7,1,1,80,0,10,0,5,1,1,1,1.1,0,0.2,-0.3,-7,-2,0,2,0.6,4.1,5.6,-1.5,13,1,6,1,2,0,2,0,12,1,3,1,1,1,2,0,1,3,1,0,1,1,5,1,1,-5,6,6,47,6,6,-33,-2,1,2,2,0,1,-200,0,400,0,0,0,-0.3,-3.9,3.6,3.4,8.6,-5.2,1,-1,2,1,0,1,-1,-1,0,-1,-2,1,-6,-3,-3,-30,34,-64,49,64,-15,2.1,2.0,0.1,1.6,1.7,-0.1,1.7,2.0,-0.3,6.9,10.5,-3.6,1,2.19,3.32,3.39
45,Serie_A,2020-02-15,1400,Lecce,Spal,0.8,0.8,0.9,0.6,4.3,5.8,3.4,2.8,-0.1,0.2,0.9,3.0,1.2,1.2,0.0,1.2,1.2,0.0,1.5,1,0,79,1,23,0,5,1,1,0,2.5,1,1.3,1,1,82,0,11,1,5,1,1,0,2.4,0,0.2,0.2,-3,12,0,0,0.1,2.1,4.4,-2.3,14,1,6,1,2,1,2,0,14,0,4,0,1,0,3,0,0,2,1,-1,2,-3,2,4,-1,-5,9,-13,9,17,-4,-48,1,1,2,1,2,-1,0,0,0,0,0,0,4.7,-1.2,5.9,0.0,0.0,0.0,1,0,1,0,0,0,-3,0,-3,10,0,10,0,3,-3,-106,-59,-47,0,6,-6,1.2,1.0,0.2,1.7,1.3,0.4,0.6,0.4,0.2,4.1,3.9,0.2,1,2.19,3.52,3.22
51,Serie_A,2020-02-16,1400,Sassuolo,Parma,1.2,1.5,1.1,0.8,5.7,6.6,4.8,3.3,0.1,0.7,0.9,3.3,1.8,1.1,0.7,1.8,1.1,0.7,1.6,1,1,85,1,12,0,9,0,2,0,1.8,0,1.3,0,0,80,1,11,1,7,0,2,0,1.9,0,0.3,0.1,5,1,2,0,-0.1,2.6,3.3,-0.7,16,1,6,1,2,0,1,0,14,1,4,0,1,0,1,0,2,2,1,0,2,-2,2,-1,-2,-2,9,-9,9,-4,-9,-9,2,1,2,1,0,-1,0,0,0,0,0,0,5.8,1.0,4.8,0.0,0.0,0.0,2,0,2,0,1,-1,2,-1,3,3,-2,5,5,-4,9,-25,-13,-12,-26,-12,-14,1.1,1.0,0.1,2.1,1.2,0.9,1.1,0.6,0.5,5.4,4.9,0.5,0,2.01,3.53,3.69
56,Serie_A,2020-02-22,1400,Bologna,Udinese,1.6,1.4,1.1,0.9,6.0,6.0,6.4,7.7,0.5,0.5,-0.4,-1.7,1.4,0.8,0.6,1.4,0.8,0.6,1.4,0,0,79,1,8,0,7,0,1,0,1.6,1,1.4,0,0,81,0,11,0,10,0,1,0,1.5,0,0.0,-0.1,-2,-3,-3,0,0.1,3.3,3.7,-0.4,15,1,4,0,2,0,2,1,19,0,5,0,0,0,1,0,-4,-1,2,1,-1,-5,-1,-1,-5,1,-4,-21,-4,-4,-79,4,-1,2,-1,2,-2,0,0,-300,0,100,0,-100,3.5,-3.6,7.1,0.0,6.7,-6.7,-1,0,-1,-1,-1,0,-1,3,-4,-3,-2,-1,0,-1,1,7,0,7,-59,-35,-24,1.9,1.8,0.1,0.7,0.9,-0.2,3.2,2.9,0.3,4.8,4.6,0.2,0,2.06,3.42,3.66
65,Serie_A,2020-03-08,1130,Parma,Spal,1.1,1.4,0.9,0.7,4.1,5.2,4.1,3.8,0.2,0.7,0.0,1.4,1.5,0.9,0.6,1.5,0.9,0.6,1.1,0,1,77,0,12,1,6,0,1,0,1.8,0,1.3,0,0,82,0,10,0,6,0,0,0,2.1,0,-0.2,0.0,-5,2,0,1,-0.3,2.8,3.0,-0.2,13,0,4,1,1,0,1,0,15,0,4,1,1,0,3,0,-2,0,0,-2,1,-5,1,5,1,-5,4,-20,4,24,4,-52,-1,1,-1,2,0,1,-100,0,-100,0,0,0,3.7,-4.1,7.8,0.0,0.0,0.0,0,0,0,0,-1,1,-3,-1,-2,0,-2,2,-7,-1,-6,-7,-34,27,-33,-38,5,0.8,1.0